In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
INPUT_DIR = '/kaggle/input/prot-3d-dataset'
OUTPUT_DIR = '/kaggle/working'

In [ ]:
import math

def is_closed_enough(residue1, residue2):
    coord1 = residue1['coord']
    coord2 = residue2['coord']

    dis = math.sqrt((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2 + (coord1[2] - coord2[2]) ** 2)
    return dis < 9.0

In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

def extract_3d_dataset(max_len='longest'):
    PROT3D_PATH = f'{INPUT_DIR}/3d_dataset.json'
    NEW_PROT3D_PATH = f'{OUTPUT_DIR}/train_set_graph.json'
    AA_INFO_PATH = f'{INPUT_DIR}/smiles_string_aa.csv'
    df = pd.read_csv(AA_INFO_PATH)
    prots = []
    with open(PROT3D_PATH, 'r') as f:
        data = json.load(f)
        for prot in tqdm(data):
            protein_length = len(prot['prot_3d'])
            from_list, to_list = [], []
            if isinstance(max_len, int) and protein_length > max_len:
                protein_length = max_len
            adj_matrix = np.zeros((protein_length, 20), dtype=np.int8)
            if protein_length != 0:
                prot_3d = prot['prot_3d']
                for i, r1 in enumerate(prot_3d[:protein_length-1]):
                    for j, r2 in enumerate(prot_3d[i + 1:protein_length]):
                        ret = is_closed_enough(r1, r2)
                        if ret:
                            idx1 = df.index[df['abbreviations'] == r1['residue']].tolist()[0]
                            idx2 = df.index[df['abbreviations'] == r2['residue']].tolist()[0]
                            from_list.append(i)
                            to_list.append(j+i+1)
                            from_list.append(j+i+1)
                            to_list.append(i)
                            adj_matrix[i, idx2] = 1
                            adj_matrix[j+i+1, idx1] = 1

                prots.append({
                    "prot_id": prot['prot_id'],
                    "kingdom": prot['organism'],
                    "label": prot['label'],
                    "partition": prot['partition'],
                    "from_list": from_list,
                    "to_list": to_list,
                    "adj_matrix": adj_matrix.tolist(),
                    "len": len(prot['prot_3d'])
                })

    with open(NEW_PROT3D_PATH, 'w') as f:
        json.dump(prots, f)

In [ ]:
extract_3d_dataset()